# Feature Importance & Business Insights (Member 3)

This notebook extracts feature importance from the trained Logistic Regression model,
identifies the top 5 churn drivers, creates a bar chart, and provides a business explanation.

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Load Saved Model & Feature Names

The model and feature names were saved by Member 2 during model training.

In [ ]:
# Load the trained Logistic Regression model
with open("../models/model.pkl", "rb") as f:
    model = pickle.load(f)

# Load feature names (column names after one-hot encoding)
with open("../models/feature_names.pkl", "rb") as f:
    feature_names = pickle.load(f)

print(f"Model type: {type(model).__name__}")
print(f"Number of features: {len(feature_names)}")
print(f"Feature names: {list(feature_names)}")

## 2. Extract Feature Importance

For Logistic Regression, we use the absolute values of `model.coef_` to measure
how strongly each feature influences the churn prediction.
A higher absolute coefficient means the feature has a bigger impact.

In [ ]:
# Extract absolute values of coefficients as importance scores
importances = np.abs(model.coef_[0])

# Create a DataFrame with feature names and their importance scores
feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
})

# Sort by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False).reset_index(drop=True)

print("\n--- All Features Ranked by Importance ---")
print(feature_importance_df.to_string(index=False))

## 3. Top 5 Most Important Features

These are the features that have the strongest influence on whether a customer churns or not.

In [ ]:
# Select top 5 features
top5 = feature_importance_df.head(5)

print("\n=== TOP 5 CHURN DRIVERS ===")
print("="*45)
for i, row in top5.iterrows():
    print(f"  {i+1}. {row['Feature']:45s}  Score: {row['Importance']:.4f}")
print("="*45)

## 4. Bar Chart — Top 5 Features Influencing Customer Churn

In [ ]:
# Set visual style
sns.set_theme(style="whitegrid")

fig, ax = plt.subplots(figsize=(10, 6))

# Create horizontal bar chart (reversed order so highest is on top)
colors = sns.color_palette("viridis", n_colors=5)
bars = ax.barh(
    top5["Feature"].values[::-1],
    top5["Importance"].values[::-1],
    color=colors[::-1],
    edgecolor="black",
    linewidth=0.5
)

# Add value labels on bars
for bar in bars:
    width = bar.get_width()
    ax.text(width + 0.01, bar.get_y() + bar.get_height()/2,
            f"{width:.4f}", va="center", fontsize=11, fontweight="bold")

ax.set_xlabel("Importance Score (|Coefficient|)", fontsize=13)
ax.set_ylabel("Feature", fontsize=13)
ax.set_title("Top 5 Features Influencing Customer Churn", fontsize=15, fontweight="bold")
plt.tight_layout()

# Save the chart
fig.savefig("../feature_importance.png", dpi=150, bbox_inches="tight")
print("Chart saved as 'feature_importance.png'")

plt.show()

## 5. Business Explanation

A plain-language interpretation of why customers are churning.

In [ ]:
business_explanation = """
=== BUSINESS EXPLANATION — Why Are Customers Churning? ===

Our analysis reveals that tenure (how long a customer has been with the company) is
by far the strongest predictor of churn. Newer customers are at much greater risk of
leaving, while long-term customers tend to stay loyal — this means the first few months
are critical for customer retention. Monthly Charges and Total Charges also play a major
role: customers paying higher monthly bills are more likely to churn, suggesting they may
feel the service is not worth the cost. Fiber Optic internet users show notably higher
churn rates compared to DSL customers, possibly due to higher pricing or unmet performance
expectations. Finally, customers on Two-Year contracts are significantly less likely to
churn, confirming that longer commitments create stronger retention.

RECOMMENDATION: The business should proactively engage new customers during their first
few months with onboarding support and loyalty incentives, offer discounts to convert
month-to-month customers to longer-term contracts, review Fiber Optic pricing and service
quality to address dissatisfaction, and consider bundling services to reduce the perceived
monthly cost for high-paying customers.
"""

print(business_explanation)